In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferMemory

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})

c:\Users\charl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\charl\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
llm = Ollama(model="llama3")

In [4]:
loader = CSVLoader('C:/Users/charl/Desktop/mRNA_codon_chart.csv', encoding="big5")
documents = loader.load()

In [ ]:
#docs = [
#    Document(page_content='變身水（Polyjuice Potion）可變成其他人的樣貌。不可拿來變身成動物，也對動物產生不了效果（包括半人半動物的生物），誤用動物毛髮的話，則會變成動物的容貌。'),
#    Document(page_content='吐真劑（Veritaserum）出自《火盃的考驗》，特徵為像水一樣清澈無味，使用者只要加入三滴，就能強迫飲用者說出真相。它是現存最強大的吐實魔藥，在《哈利波特》的虛構世界觀中受英國魔法部嚴格控管。J·K·羅琳表示，吐真劑最適合用在毫無戒心、易受傷害、缺乏自保技能的人身上，有些巫師能使用鎖心術等方式保護自己免受吐真劑影響。'),
#    Document(page_content='福來福喜（Felix Felicix）出自《混血王子》，是一種稀有而且難以調製的金色魔藥，能夠給予飲用者好運。魔藥的效果消失之前，飲用者的所有努力都會成功。假如飲用過量，會導致頭暈、魯莽和危險的過度自信，甚至成為劇毒。'),
#]

In [5]:
vectorstore = Chroma.from_documents(documents, embeddings)

c:\Users\charl\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
retriever = vectorstore.as_retriever()

In [47]:
query = "TTT,WASD,TTT,ATT"

In [48]:
system_prompt = (
    "以,符號來分割"
    "只參考表格DNA欄位有出現的codon進行檢索和對應答案"
    "請嚴格檢查序列不要錯誤，沒有在原本序列出現的都是幻覺，請不要出現在最終結果"
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query})

{'input': 'TTT,WASD,TTT,ATT',
 'context': [Document(page_content='DNA: WASD\nCodon: WASD\nAmino Acid: TEST\nSingle Letter: TEST\nDescription: 每三個為一組，若剛好三個為WASD，則對應的RNA codon為WASD，因此胺基酸為TEST，代號為TEST', metadata={'row': 64, 'source': 'C:/Users/charl/Desktop/mRNA_codon_chart.csv'}),
  Document(page_content='DNA: TTT\nCodon: UUU\nAmino Acid: Phe\nSingle Letter: F\nDescription: 每三個為一組，若剛好三個為TTT，則對應的RNA codon為UUU，因此胺基酸為Phe，代號為F', metadata={'row': 0, 'source': 'C:/Users/charl/Desktop/mRNA_codon_chart.csv'}),
  Document(page_content='DNA: GAT\nCodon: GAU\nAmino Acid: Asp\nSingle Letter: D\nDescription: 每三個為一組，若剛好三個為GAT，則對應的RNA codon為GAU，因此胺基酸為Asp，代號為D', metadata={'row': 44, 'source': 'C:/Users/charl/Desktop/mRNA_codon_chart.csv'}),
  Document(page_content='DNA: TTC\nCodon: UUC\nAmino Acid: Phe\nSingle Letter: F\nDescription: 每三個為一組，若剛好三個為TTC，則對應的RNA codon為UUC，因此胺基酸為Phe，代號為F', metadata={'row': 1, 'source': 'C:/Users/charl/Desktop/mRNA_codon_chart.csv'})],
 'answer': 'Based on the provided table,

#### 20240618 測試結果，直接處理好DNA序列，以逗號來分割最簡單，給完整序列很難給出正確答案。

In [17]:
memory = ConversationBufferMemory()

In [22]:
memory.clear()

### //////